In [1]:
from selenium import webdriver
import time

def login_to_ancestry(username, password):
    driver.get('https://www.ancestry.com/account/signin/')
    time.sleep(3)

    frame = driver.find_element_by_id('signInFrame')
    driver.switch_to.frame(frame)

    username_field = driver.find_element_by_id('username')
    password_field = driver.find_element_by_id('password')

    username_field.send_keys(username)
    password_field.send_keys(password)
    time.sleep(3)
    driver.find_element_by_id('signInBtn').click()
    time.sleep(3)

In [27]:
def get_match_data(test_id, group=None, note=None):
    root_url = 'https://www.ancestry.com/discoveryui-matches/match-list/'
    driver.get(f'{root_url}{test_id}')
    time.sleep(3)
    
    if group:
        driver.find_element_by_id('filter_groups').click()
        time.sleep(1)
        options = driver.find_element_by_xpath(
            '//*[@class="calloutMenu groupList ng-star-inserted"]')
        group_button = options.find_element_by_xpath(
            f'.//*[@title="{group}"]')
        driver.execute_script("arguments[0].scrollIntoView();", group_button)
        group_button.click()
        time.sleep(1)
        driver.find_element_by_id('applyGroups').click()
        time.sleep(3)
        
    if note:
        driver.find_element_by_id('filter_notes').click()
        time.sleep(3)
        
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script(
            "window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        new_height = driver.execute_script(
            "return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
    match_list = driver.find_element_by_xpath(
        '//*[starts-with(@class, "match-list-body")]')
    matches = match_list.find_elements_by_xpath(
        './/*[starts-with(@class, "matchGrid")]')

    match_dict = {}
    for match in matches:
        match_id = match.find_element_by_xpath(
            './/*[starts-with(@href, '
            '"/discoveryui-matches/compare/")]').get_attribute(
            'href').replace('https://www.ancestry.com/discoveryui-matches/'
                            f'compare/{test_id.upper()}/with/', '')
        match_dict[match_id] = {}
        data_list = match.text.split('\n')
        if data_list[1].startswith('Managed by '):
            data_list[0] = f'{data_list[0]} {data_list[1]}'
            del data_list[1]
        match_dict[match_id]['match_name'] = data_list[0]
        match_dict[match_id]['relation'] = data_list[1]
        match_dict[match_id]['cMs'] = int(data_list[2].split(' cM across ')[0].replace(
            'Shared DNA: ', '').replace(',', '').strip())
        match_dict[match_id]['segments'] = int(data_list[2].split(
            ' cM across ')[1].replace(' segments', '').strip())
        match_dict[match_id]['tree'] = data_list[3]
        if data_list[4] == 'Common ancestor':
            match_dict[match_id]['common_ancestor'] = True
            del data_list[4]
        else:
            match_dict[match_id]['common_ancestor'] = False
        if data_list[4] == 'Add/edit groups':
            match_dict[match_id]['grouped'] = True
        else:
            match_dict[match_id]['grouped'] = False
        del data_list[0:5]
        match_dict[match_id]['note'] = ' '.join(data_list)
        
        if note not in match_dict[match_id]['note']:
            del match_dict[match_id]
            continue
        
        groups_area = match.find_element_by_xpath('.//*[starts-with(@class, '
                                                  '"indicatorGroupCollection")]')
        groups = groups_area.find_elements_by_xpath(
            './/*[starts-with(@class, "icon iconStar")]|'
            './/*[starts-with(@class, "indicatorGroup")]')
        match_dict[match_id]['groups'] = [group.get_attribute(
            'title') for group in groups]
    return match_dict

In [28]:
import json

def get_pedigree(test_id, match_id, file_path):
    root_url = 'https://www.ancestry.com/discoveryui-matches/compare/'
    driver.get(f'{root_url}{test_id}/with/{match_id}')
    time.sleep(3)
    
    private = driver.find_elements_by_xpath(
        "//*[@class='icon iconLock ng-star-inserted']")
    if len(private) > 0:
        return
    
    unavailable = driver.find_elements_by_xpath(
        "//*[@class='descriptionDesktop flex-item ng-star-inserted']|"
        "//*[@class='descriptionMobile flex-item ng-star-inserted']")
    for element in unavailable:
        if element.text.startswith('Our backend services are overtaxed'):
            return
    
    driver.find_element_by_id("expandButton").click()
    time.sleep(2)

    ancestors = driver.find_elements_by_xpath(
        "//*[@class='father ng-star-inserted']|"
        "//*[@class='mother ng-star-inserted']")

    html_dict = {}
    for ancestor in ancestors:
        if ancestor.get_attribute('class') == 'father ng-star-inserted':
            new_bit = '0'
        elif ancestor.get_attribute('class') == 'mother ng-star-inserted':
            new_bit = '1'
        inner_html = ancestor.get_attribute('innerHTML')
        character_counter = {}
        for key in html_dict:
            if inner_html in html_dict[key]:
                character_counter[key] = len(html_dict[key])
        if len(character_counter) == 0:
            key = '1' + new_bit
        else:
            child = [key for key in character_counter if 
                     character_counter[key] == min(character_counter.values())][0]
            key = child + new_bit
        html_dict[key] = inner_html

    cleaned_html_dict = {}
    for ancestor in html_dict:
        cleaned_html = html_dict[ancestor]
        parents = [key for key in html_dict if key.startswith(ancestor) and 
                   len(key) == len(ancestor) + 1]
        for parent in parents:
            cleaned_html = cleaned_html.replace(html_dict[parent], '')
        cleaned_html_dict[ancestor] = cleaned_html

    pedigree = {}
    pedigree['1'] = {}
    root = driver.find_element_by_xpath("//*[@class='card root gen1']")
    rootHTML = root.get_attribute('innerHTML')
    forename = rootHTML.split(
        '<h6 class="userCardTitle textClipped ng-star-inserted">')[1].split(
        ' <span')[0]
    if forename.startswith('Private</h6>'):
        pedigree['1']['Private'] = True
        pedigree['1']['visited'] = False
    else:
        surname = rootHTML.split(
            f'<h6 class="userCardTitle textClipped ng-star-inserted">{forename} <span')[
            1].split('>')[1].split('</span')[0]
        pedigree['1']['Forename'] = forename
        pedigree['1']['Surname'] = surname
        pedigree['1']['Private'] = False
        pedigree['1']['visited'] = False
    for ancestor in cleaned_html_dict:
        pedigree[ancestor] = {}
        if cleaned_html_dict[ancestor] == '':
            pedigree[ancestor]['Private'] = True
            pedigree[ancestor]['visited'] = False
        else:
            name_html = cleaned_html_dict[ancestor].split(
                '<strong>')[1].split('</strong>')[0].replace('</span>', '')
            if ' <span' in name_html:
                forename = name_html.split(' <span')[0]
                surname = name_html.split(' <span')[1].split('>')[1]
                if (forename == '') and (surname == ''):
                    pedigree[ancestor]['Private'] = True
                    pedigree[ancestor]['visited'] = False
                else:
                    pedigree[ancestor]['Forename'] = forename
                    pedigree[ancestor]['Surname'] = surname
                    pedigree[ancestor]['Private'] = False
                    pedigree[ancestor]['visited'] = False
            else:
                pedigree[ancestor]['Private'] = True
                pedigree[ancestor]['visited'] = False

    driver.find_element_by_xpath("//*[starts-with(@class, 'closeBtn')]").click()
    time.sleep(2)

    root = driver.find_element_by_xpath("//*[@class='card root gen1']")
    rootHTML = root.get_attribute('innerHTML')
    if '<div class="userCardDates ng-star-inserted">' in rootHTML:
        lifespan = rootHTML.split(
            f'<div class="userCardDates ng-star-inserted">')[
            1].split('</div>')[0]
        pedigree['1']['Lifespan'] = lifespan

    root_keys = [key for key in pedigree if (
        pedigree[key]['Private'] == False) and ((key[:-1] == '') or (
        pedigree[key[:-1]]['Private'] == True))]
    if len(root_keys) == 0:
        return

    cards = driver.find_elements_by_xpath(
        './/*[starts-with(@class, "card root gen1")]|'
        './/*[starts-with(@class, "P card gen")]|'
        './/*[starts-with(@class, "M card gen")]')
    for root_key in root_keys:
        root_forename = pedigree[root_key]['Forename']
        root_surname = pedigree[root_key]['Surname']
        lifespan_found = False
        for card in cards:
            if (root_forename in card.get_attribute('innerHTML')) and (
                root_surname in card.get_attribute('innerHTML')):
                if (f'<div class="userCardDates ng-star-inserted">' in 
                    card.get_attribute('innerHTML')):
                    root_lifespan = card.get_attribute('innerHTML').split(
                        f'<div class="userCardDates ng-star-inserted">')[1].split(
                        '</div>')[0]
                    pedigree[root_key]['Lifespan'] = root_lifespan
                    lifespan_found = True
                else:
                    root_lifespan = ''
                    pedigree[root_key]['Lifespan'] = root_lifespan
                if root_key == root_keys[0]:
                    current_key = root_key
                    forename = root_forename
                    surname = root_surname
                    lifespan = root_lifespan
                    root_button = card
                break
        if lifespan_found == False:
            root_lifespan = ''
            pedigree[root_key]['Lifespan'] = root_lifespan

    root_button.click()
    time.sleep(2)

    while True:
        current_card = driver.find_element_by_xpath(
            "//*[starts-with(@class, 'userCardTitle ng-tns-c74-0')]").text
        if (forename not in current_card) or (surname not in current_card):
            close = driver.find_element_by_xpath(
                "//*[starts-with(@class, 'closeBtn')]")
            close.click()
            time.sleep(2)
            root_key = sorted(
                [root for root in root_keys if current_key.startswith(root)], 
                key=lambda x: len(x), reverse=True)[0]
            root_forename = pedigree[root_key]['Forename']
            root_surname = pedigree[root_key]['Surname']
            root_lifespan = pedigree[root_key]['Lifespan']
            if len(root_key) <=5:
                cards = driver.find_elements_by_xpath(
                    './/*[starts-with(@class, "card root gen1")]|'
                    './/*[starts-with(@class, "P card gen")]|'
                    './/*[starts-with(@class, "M card gen")]')
                for card in cards:
                    if (root_forename in card.get_attribute('innerHTML')) and (
                        root_surname in card.get_attribute('innerHTML')) and (
                        root_lifespan in card.get_attribute('innerHTML')):
                        root_button = card
                        break
                root_button.click()
                time.sleep(2)
            elif len(root_key) > 5:
                driver.find_element_by_id("expandButton").click()
                time.sleep(2)
                innerHTML = cleaned_html_dict[root_key]
                ancestors = driver.find_elements_by_xpath(
                    "//*[@class='father ng-star-inserted']|"
                    "//*[@class='mother ng-star-inserted']")
                root_ancestor = sorted(
                    [ancestor for ancestor in ancestors if innerHTML in 
                     ancestor.get_attribute('innerHTML')], 
                    key=lambda x: len(x.get_attribute('innerHTML')))[0]
                root_button = root_ancestor.find_element_by_xpath(
                    ".//*[@class='userCardTitle textClipped ng-star-inserted']")
                root_button.click()
                time.sleep(2)
            for i in current_key[len(root_key):]:
                parent_section = driver.find_elements_by_xpath(
                    "//*[starts-with(@class, 'topSpacing ng-tns-c74-0')]"
                    "[./*[contains(text(), 'Parents')]]")

                if len(parent_section) > 0:
                    parent_section = parent_section[0]
                    parents = parent_section.find_elements_by_xpath(
                        ".//*[starts-with(@class, 'card ng-tns-c74-0')]")
                    if len(parents) > 1:
                        parent = parents[int(i)]
                    elif len(parents) == 1:
                        parent = parents[0]
                    driver.execute_script(
                        "arguments[0].scrollIntoView();", parent)
                    parent.click()
                    time.sleep(2)
                    
        if pedigree[current_key]['visited'] == False:
            pedigree[current_key]['visited'] = True
            vitals = driver.find_elements_by_xpath(
                "//*[@class='con conVitals ng-tns-c74-0 ng-star-inserted']")
            if len(vitals) > 0:
                vitals = vitals[0]
                vitals = vitals.get_attribute('outerHTML').replace(
                    '<div class="con conVitals ng-tns-c74-0 ng-star-inserted">'
                    '<div class="ng-tns-c74-0 ng-star-inserted">', '')

                events = vitals.split('<span class="bold ng-tns-c74-0">')[1:]
                for eventHTML in events:
                    event = eventHTML.split('</span>')[0]
                    if event in pedigree[current_key]:
                        counter = 2
                        while True:
                            if event + f' {counter}' not in pedigree[current_key]:
                                event = event + f' {counter}'
                                break
                            else:
                                counter += 1
                    pedigree[current_key][event] = {'Date': None, 'Location': None}
                    if '<span class="bullet ng-tns-c74-0 ng-star-inserted"' in eventHTML: 
                        date = eventHTML.split('<br class="ng-tns-c74-0"> ')[1].split(
                            '<span class="bullet ng-tns-c74-0 ng-star-inserted"')[0]
                        location = eventHTML.split(
                            '<span class="ng-tns-c74-0 ng-star-inserted">')[1].split(
                            '</span><!----></div>')[0]
                        pedigree[current_key][event]['Date'] = date
                        pedigree[current_key][event]['Location'] = location
                    else:
                        if eventHTML.startswith(
                            f'{event}</span><br class="ng-tns-c74-0">'
                            ' <!----><span class="ng-tns-c74-0 ng-star-inserted">'):
                            location = eventHTML.split(
                                '<span class="ng-tns-c74-0 ng-star-inserted">')[1].split(
                                '</span><!----></div>')[0]
                            pedigree[current_key][event]['Location'] = location
                        else:
                            date = eventHTML.split(
                                '<br class="ng-tns-c74-0"> ')[1].split(
                                '<!----><!----></div>')[0]
                            pedigree[current_key][event]['Date'] = date

            parent_section = driver.find_elements_by_xpath(
                "//*[starts-with(@class, 'topSpacing ng-tns-c74-0')]"
                "[./*[contains(text(), 'Parents')]]")

            if (len(parent_section) > 0) and (len(current_key) <= 12):
                parent_section = parent_section[0]
                parents = parent_section.find_elements_by_xpath(
                    ".//*[starts-with(@class, 'card ng-tns-c74-0')]")

                counter = 0
                for parent in parents:
                    parentHTML = parent.get_attribute('innerHTML')
                    forename = parentHTML.split(
                        '<h6 class="userCardTitle textClipped ng-star-inserted">')[1].split(
                        ' <span')[0]
                    if forename.startswith('Private</h6>'):
                        private = True
                    else:
                        surname = parentHTML.split(
                            f'<h6 class="userCardTitle textClipped ng-star-inserted">{forename} <span')[
                            1].split('>')[1].split('</span')[0]
                        lifespan = parentHTML.split(
                            f'<div class="userCardDates ng-star-inserted">')[
                            1].split('</div>')[0]
                        private = False

                    if len(parents) > 1:
                        parent_key = current_key + str(counter)
                    if len(parents) == 1:
                        if (current_key + '0') in pedigree:
                            parent_key = (current_key + '0')
                        elif (current_key + '1') in pedigree:
                            parent_key = (current_key + '1')
                        else:
                            parent_key = current_key + str(counter)
                    pedigree[parent_key] = {}
                    if private == False:
                        pedigree[parent_key]['Forename'] = forename
                        pedigree[parent_key]['Surname'] = surname
                        pedigree[parent_key]['Lifespan'] = lifespan
                    pedigree[parent_key]['Private'] = private
                    pedigree[parent_key]['visited'] = False
                    counter += 1

        unexplored_ancestors = [ancestor for ancestor in pedigree if 
                                (pedigree[ancestor]['visited'] == False) and (
                                    pedigree[ancestor]['Private'] == False)]

        if len(unexplored_ancestors) == 0:
            break

        else:
            unexplored_from_here = [
                ancestor for ancestor in unexplored_ancestors if 
                ancestor.startswith(current_key) and ancestor not in 
                root_keys]

        if len(unexplored_from_here) > 0:
            current_key = unexplored_from_here[0]
            forename = pedigree[current_key]['Forename']
            surname = pedigree[current_key]['Surname']
            if lifespan in pedigree[current_key]:
                lifespan = pedigree[current_key]['Lifespan']
            else:
                lifespan = ''

            parent_section = driver.find_elements_by_xpath(
                "//*[starts-with(@class, 'topSpacing ng-tns-c74-0')]"
                "[./*[contains(text(), 'Parents')]]")

            if (len(parent_section) > 0):
                parent_section = parent_section[0]
                parents = parent_section.find_elements_by_xpath(
                    ".//*[starts-with(@class, 'card ng-tns-c74-0')]")

                for parent in parents:
                    parentHTML = parent.get_attribute('innerHTML')
                    if (forename in parentHTML) and (
                        surname in parentHTML) and (
                        lifespan in parentHTML):
                        driver.execute_script("arguments[0].scrollIntoView();", parent)
                        parent.click()
                        time.sleep(2)
                        break
                    else:
                        continue

        elif len(unexplored_from_here) == 0:
            if current_key[:-1] == '':
                driver.find_element_by_xpath(
                    "//*[starts-with(@class, 'closeBtn')]").click()
                time.sleep(2)
                unexplored_root_keys = [key for key in root_keys if key in 
                                        unexplored_ancestors]
                if len(unexplored_root_keys) > 0:
                    root_key = unexplored_root_keys[0]
                else:
                    root_key = [key for key in root_keys if len(
                        [ancestor for ancestor in unexplored_ancestors if 
                         ancestor.startswith(key)]) > 0][0]
                root_forename = pedigree[root_key]['Forename']
                root_surname = pedigree[root_key]['Surname']
                root_lifespan = pedigree[root_key]['Lifespan']
                if len(root_key) <=5:
                    cards = driver.find_elements_by_xpath(
                        './/*[starts-with(@class, "card root gen1")]|'
                        './/*[starts-with(@class, "P card gen")]|'
                        './/*[starts-with(@class, "M card gen")]')
                    for card in cards:
                        if (root_forename in card.get_attribute('innerHTML')) and (
                            root_surname in card.get_attribute('innerHTML')) and (
                            root_lifespan in card.get_attribute('innerHTML')):
                            root_button = card
                            break
                    root_button.click()
                    time.sleep(2)
                elif len(root_key) > 5:
                    driver.find_element_by_id("expandButton").click()
                    time.sleep(2)
                    innerHTML = cleaned_html_dict[root_key]
                    ancestors = driver.find_elements_by_xpath(
                        "//*[@class='father ng-star-inserted']|"
                        "//*[@class='mother ng-star-inserted']")
                    root_ancestor = sorted(
                        [ancestor for ancestor in ancestors if innerHTML in 
                         ancestor.get_attribute('innerHTML')], 
                        key=lambda x: len(x.get_attribute('innerHTML')))[0]
                    root_button = root_ancestor.find_element_by_xpath(
                        ".//*[@class='userCardTitle textClipped ng-star-inserted']")
                    root_button.click()
                    time.sleep(2)
                current_key = root_key
                forename = root_forename
                surname = root_surname
                lifespan = root_lifespan
                time.sleep(2)
                
            elif pedigree[current_key[:-1]]['Private'] == False:
                current_key = current_key[:-1]
                forename = pedigree[current_key]['Forename']
                surname = pedigree[current_key]['Surname']
                if 'Lifespan' in pedigree[current_key]:
                    lifespan = pedigree[current_key]['Lifespan']
                else:
                    lifespan = ''

                spouse_and_children_section = driver.find_elements_by_xpath(
                    "//*[starts-with(@class, 'topSpacing ng-tns-c74-0')]"
                    "[./*[contains(text(), 'Spouse and Children')]]")
                
                if len(spouse_and_children_section) > 0:
                    spouse_and_children_section = spouse_and_children_section[0]
                    
                    spouse_and_children = spouse_and_children_section.find_elements_by_xpath(
                        ".//*[starts-with(@class, 'card ng-tns-c74-0')]")

                    for spouse_and_child in spouse_and_children:
                        spouse_and_childHTML = spouse_and_child.get_attribute('innerHTML')
                        if (forename in spouse_and_childHTML) and (
                            surname in spouse_and_childHTML) and (
                            lifespan in spouse_and_childHTML):
                            driver.execute_script("arguments[0].scrollIntoView();", spouse_and_child)
                            spouse_and_child.click()
                            time.sleep(2)
                            break
                        else:
                            continue
                else:
                    continue

            elif pedigree[current_key[:-1]]['Private'] == True:
                driver.find_element_by_xpath(
                    "//*[starts-with(@class, 'closeBtn')]").click()
                time.sleep(2)
                unexplored_root_keys = [key for key in root_keys if key in 
                                        unexplored_ancestors]
                if len(unexplored_root_keys) > 0:
                    root_key = unexplored_root_keys[0]
                else:
                    root_key = [key for key in root_keys if len(
                        [ancestor for ancestor in unexplored_ancestors if 
                         ancestor.startswith(key)]) > 0][0]
                root_forename = pedigree[root_key]['Forename']
                root_surname = pedigree[root_key]['Surname']
                root_lifespan = pedigree[root_key]['Lifespan']
                if len(root_key) <=5:
                    cards = driver.find_elements_by_xpath(
                        './/*[starts-with(@class, "card root gen1")]|'
                        './/*[starts-with(@class, "P card gen")]|'
                        './/*[starts-with(@class, "M card gen")]')
                    for card in cards:
                        if (root_forename in card.get_attribute('innerHTML')) and (
                            root_surname in card.get_attribute('innerHTML')) and (
                            root_lifespan in card.get_attribute('innerHTML')):
                            root_button = card
                            break
                    root_button.click()
                    time.sleep(2)
                elif len(root_key) > 5:
                    driver.find_element_by_id("expandButton").click()
                    time.sleep(2)
                    innerHTML = cleaned_html_dict[root_key]
                    ancestors = driver.find_elements_by_xpath(
                        "//*[@class='father ng-star-inserted']|"
                        "//*[@class='mother ng-star-inserted']")
                    root_ancestor = sorted(
                        [ancestor for ancestor in ancestors if innerHTML in 
                         ancestor.get_attribute('innerHTML')], 
                        key=lambda x: len(x.get_attribute('innerHTML')))[0]
                    root_button = root_ancestor.find_element_by_xpath(
                        ".//*[@class='userCardTitle textClipped ng-star-inserted']")
                    root_button.click()
                    time.sleep(2)
                current_key = root_key
                forename = root_forename
                surname = root_surname
                lifespan = root_lifespan
                time.sleep(2)

    cleaned_pedigree = {}
    for ancestor in pedigree:
        cleaned_pedigree[ancestor] = {}
        if 'Forename' in pedigree[ancestor]:
            cleaned_pedigree[ancestor]['Forename'] = pedigree[
                ancestor]['Forename'].replace('&amp;', '&')
        if 'Surname' in pedigree[ancestor]:
            cleaned_pedigree[ancestor]['Surname'] = pedigree[
                ancestor]['Surname'].replace('&amp;', '&')
        if pedigree[ancestor]['Private'] == True:
            cleaned_pedigree[ancestor]['Forename'] = 'Private'
            cleaned_pedigree[ancestor]['Surname'] = 'Private'
        for key in pedigree[ancestor]:
            if key not in [
                'Forename', 'Surname', 'Lifespan', 'Private', 'visited']:
                cleaned_pedigree[ancestor][key] = pedigree[ancestor][key]

    pedigree_json = json.dumps(cleaned_pedigree)
    with open(f'{file_path}/{match_id}.json', 'w') as file:
        file.write(pedigree_json)